# EFM Demo

This demo is recorded as a video and is available on [YouTube](https://www.youtube.com/watch?v=mWjnNM5g2YU&t=24s)

| |Hostname|Role|
| :-: | :-: | :-: |
|VM1|pg1|Master|
|VM2|pg2|Standby|
|VM3|pg3|Client|

## Demo prep

### Prepare demo environment

In [5]:
tpaexec provision .


PLAY [Provision cluster] *******************************************************

TASK [Require cluster directory to be specified] *******************************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}

TASK [Set tpa_dir] *************************************************************
ok: [localhost]

TASK [Set full path to cluster_dir and config_file] ****************************
ok: [localhost]

TASK [Load cluster configuration file] *****************************************
ok: [localhost]

TASK [set_fact] ****************************************************************
ok: [localhost]

TASK [Retrieve hash of last provisioned config.yml, if available] **************
ok: [localhost]

TASK [Compute hash of config.yml at present] ***********************************
ok: [localhost]

TASK [Exit if there are no changes to config.yml since the last provision] *****
ok: [localhost]

PLAY RECAP **********************************************************

In [6]:
tpaexec deploy .


PLAY [Provision cluster] *******************************************************

TASK [Require cluster directory to be specified] *******************************
ok: [localhost] => {
    "changed": false,
    "msg": "All assertions passed"
}

TASK [Set tpa_dir] *************************************************************
ok: [localhost]

TASK [Set full path to cluster_dir and config_file] ****************************
ok: [localhost]

TASK [Load cluster configuration file] *****************************************
ok: [localhost]

TASK [set_fact] ****************************************************************
ok: [localhost]

TASK [Retrieve hash of last provisioned config.yml, if available] **************
ok: [localhost]

TASK [Compute hash of config.yml at present] ***********************************
ok: [localhost]

TASK [Exit if there are no changes to config.yml since the last provision] *****
ok: [localhost]

PLAY RECAP **********************************************************

In [8]:
tpaexec show-password . enterprisedb

1Pwd%19rK#sS5XhxzGPA6uA!OOQoxXcA


#### Test connections

Open three terminal windows and position them vertically.

In each terminal window run `ssh -F ssh_config <pg1|pg2|pg3> "uname -n"`

Prepare demo content (create table EMP, create replication slots and changes efm settings).

In [13]:
./demo_prep.sh

data.sql                                      100% 1996     3.7MB/s   00:00    
pg1.sh                                        100%  222   213.7KB/s   00:00    
Connection to 192.168.17.11 closed.
Connection to 192.168.17.11 closed.
/var/lib/edb-as/.bashrc: line 1: ulimit: core file size: cannot modify limit: Operation not permitted
CREATE TABLE
CREATE TABLE
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
Connection to 192.168.17.11 closed.
pg3.sh                                        100%  191   268.2KB/s   00:00    
Connection to 192.168.17.14 closed.
Removed /etc/systemd/system/multi-user.target.wants/edb-efm-4.7.service.
Connection to 192.168.17.14 closed.
pg1
remote_ctl.sh                                 100%  751     1.5MB/s   00:00    
efm_rewind.sh                                 100% 4010     5.6MB/s   00:00    
efm_reconfigure_n

: 1

## Demo flow

Create three terminal windows and SSH into each server.

|pg1 - Master|pg2 - Standby|pg3 - Client|
|:---:|:---:|:---:|
|`ssh -F ssh_config pg1`|`ssh -F ssh_config pg2`|`ssh -F ssh_config pg3`|

Start monitoring the cluster on pg2


|pg1 - Master|pg2 - Standby|pg3 - Client|
| :-: | :-: | :-: |
||`sudo watch -n 1 /usr/edb/efm-4.7/bin/efm cluster-status efmdemo`||


As we can see, the master and standby databases are online and synchronized: the WAL LSN number is the same on both: primary and standby databases.

In Terminal 3 I connect to both databases using psql. The read-write attribute defines that I always connect to a primary database because only the master database can receive and process the write transactions.

We can query the emp table. And check if we are on the master:


|pg1 - Master|pg2 - Standby|pg3 - Client|
| :---: | :---: | :---: |
|||<p>`sudo -i -u enterprisedb psql postgresql://pg1,pg2/edb?target_session_attrs=read-write`</p><p>`SELECT * from EMP;`</p><p>`SELECT inet_server_addr();`</p>|

## Failover Demo

Shut down pg1.


|pg1 - Master|pg2 - Standby|pg3 - Client|
| :-: | :-: | :-: |
|`sudo systemctl stop postgres.service`|***EFM Monitor Session running***||

In Terminal 3, execute the query again.

|pg1 - Standby|pg2 - Master|pg3 - Client|
| :-: | :-: | :-: |
||***EFM Monitor Session***|<p>`SELECT inet_server_addr();`</p><p>`SELECT * from EMP;`</p>|

Recover master database (rephrase "But the situation is not good. The old master database does not exist. And we only have one database running without any protection.")

This script starts the old primary database as the new standby. The tool `pg_rewind` can be used for this. 

|pg1 - Standby|pg2 - Master|pg3 - Client|
| :-: | :-: | :-: |
|`sudo /usr/edb/efm-4.7/bin/efm_reconfigure_node.sh`|***EFM Monitor Session***||

In case WAL replication may takes too long, force a WAL switch.


|pg1 - Standby|pg2 - Master|pg3 - Client|
| :-: | :-: | :-: |
||***EFM Monitor Session***|`select pg_switch_wal();`|

## Switchover

|pg1 - Standby|pg2 - Master|pg3 - Client|
| :-: | :-: | :-: |
|`sudo /usr/edb/efm-4.4/bin/efm promote efmdemo -switchover`|***EFM Monitor Session***||

Result:

|pg1 - Master|pg2 - Standby|pg3 - Client|
| :-: | :-: | :-: |
||***EFM Monitor Session***||



## Demo cleanup

In [ ]:
tpaexec deprovision .